# EKE preprocessing

This notebook shows how to generate the EKE_timeseries.nc file

In [1]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [2]:
from dask.distributed import Client

In [3]:
from utils import area,ccrs_land,add_patches
import datetime

In [4]:
c = Client()
c

Client Scheduler: tcp://127.0.0.1:33065 Dashboard: /proxy/8787/status,Cluster Workers: 8 Cores: 32 Memory: 137.44 GB


In [5]:
path_v4 = '/g/data/ua8/CMEMS_SeaLevel/v4-0/'
inputfiles=[path_v4+"{0}/".format(ii) for ii in range(1993,2018)] 
inputfiles.append('/g/data/ua8/CMEMS_SeaLevel/v2019/2018/')
inputfiles.append('/g/data/ua8/CMEMS_SeaLevel/v2019/2019/')
import os
files=[]
for folder in inputfiles:
    for f in os.listdir(folder):
        if '.nc' in f:
            files.append(os.path.join(folder, f))
            
# inputfiles='path2AVISO+/*.nc'

In [ ]:
dataset_AVISO = xr.open_mfdataset(files)#,parallel=True)

In [ ]:
dataset_AVISO

### Compute EKE:

\begin{equation}
EKE = \frac{1}{2} \left(u'^2+v'^2\right)
\end{equation}

The provided $u'$ and $v'$ from AVISO is an anomaly from 1993-2005. Changing this period does not affect the main result of the study.

In [ ]:
EKE = 1/2 * (dataset_AVISO.ugosa**2 + dataset_AVISO.vgosa**2)

Rechunk dataset

In [ ]:
EKE_rechunk = EKE.chunk({"latitude": 100, "longitude": 100,'time':365})

In [ ]:
EKE_rechunk

Coarsen dataset to a 1 degree grid.

In [ ]:
EKE_coarsen = EKE_rechunk.coarsen({'latitude':4,'longitude':4}).mean()

Roll over 12 months:

In [ ]:
EKE_rolled = EKE_coarsen.rolling(time=365,center=True).mean().compute() #This computation requires ~ 50 GB of RAM.

In [ ]:
EKE_rolled = EKE_rolled.to_dataset(name="EKE").rename({'latitude':'lat','longitude':'lon'})

In [ ]:
EKE_rolled

### Add metadata

In [ ]:
EKE_rolled.attrs['title'] = "Eddy Kinetic Energy"
EKE_rolled.attrs['Description'] = """Eddy Kinetic Energy computed from AVISO+ altimetry products."""
EKE_rolled.attrs['Publication'] = "Dataset created for Martínez-Moreno, J. et. al. 2020: \n 'Global changes in oceanic mesoscale currents over the satellite altimetry record'"
EKE_rolled.attrs['Author'] = "Josué Martínez-Moreno"
EKE_rolled.attrs['Contact'] = "josue.martinezmoreno@anu.edu.au"

EKE_rolled.attrs['Created date'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

counter = 0
units = [r"$m^2s^{-2}$"]
names = ["EKE"]
long_names = ["Eddy Kinetic Energy"]

EKE_rolled['EKE'].attrs['units'] = units[counter]
EKE_rolled['EKE'].attrs['name'] = names[counter]
EKE_rolled['EKE'].attrs['long_name'] = long_names[counter]

EKE_rolled['EKE'].attrs['missing_value'] = np.nan
EKE_rolled['EKE'].attrs['valid_min'] = np.nanmin(EKE_rolled['EKE'])
EKE_rolled['EKE'].attrs['valid_max'] = np.nanmax(EKE_rolled['EKE'])
EKE_rolled['EKE'].attrs['valid_range'] = [np.nanmin(EKE_rolled['EKE']),np.nanmax(EKE_rolled['EKE'])]

## Store netCDF

In [ ]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in EKE_rolled.data_vars}

EKE_rolled.to_netcdf('../datasets/EKE_timeseries.nc', encoding=encoding)